# Exercise: Data Science for Responsible Decision Making


## Introduction

Please refer to the instructions provided in the exercise description (.pptx file) as the source of truth. You may this notebook as a guide to attempt the exercises in a Jupyter Notebook format using the tRBS open-source code package.

### Summarized instructions:  
 1. **Start with the assignment description:** Carefully read the exercise description instructions (in the .pptx file).  <br><br>
2. **Prepare the template:** Follow the instructions in the exercise description (Slide 13). To help you get started, we have prepared a starter template for you under **StarterTemplate.xlsx**.  
	   i. Fill in the missing fields in the sheets `key_outputs`, `decision_makers_options` and `theme_weights`.   <br><br>
 3. **Save the file:** After you have prepared the template, save it in **data/StarterTemplate/xlsx/**.  <br><br>
 4. **Run through the notebook** and note your observations.  <br><br>

----
**After** you have run through the notebook once:

 5. **Make your changes** in the **theme weights** (for the economic growth theme) in the template as per the instructions (slide 13), re-run the notebook and note the differences in your results.  <br><br>
 6. **Add the relationship** (see slide 14) between the investment in clean drinking water and the return on investment (in `scenarios`) and the income increase for low income families (in `fixed_inputs`) along with their respective dependencies (in `dependencies`) and **re-run through the notebook**.  <br><br>
 7. **Change the weights** (see slide 15) in `scenario_weights`, `theme_weights` or `key_output_weights` as per your preferences and re-run the analyses and note the differences in your results.


### (Ideas for extra data science)
- Change the weights from within the python code
- Fix the constraint with the DMO in the code
- Some error (DONT PANIC)

  
### `TheResponsibleBusinessSimulator` class  
In order to work with the Responsible Business Simulator model, we will work with the ```TheResponsibleBusinessSimulator``` class. This class can be found in **core/trbs.py**. This class takes three input values: (i) the folder where data of the case can be found, (ii) the format of the data (options: xlsx, json, csv) and (iii) the name of the case.





In [ ]:
from pathlib import Path
from core.trbs import TheResponsibleBusinessSimulator

# Specify your case and format. 
path = Path.cwd() / 'data'
file_format = 'xlsx'
name = 'FinalTemplate'

case = TheResponsibleBusinessSimulator(path, file_format, name)

Let's see how ```case``` looks currently:

In [ ]:
print(case)

Currently, it is rather empty. To get the input data, we will first need to ```build``` our case.

In [ ]:
case.build()
print(case)

### (Optional) Change the parameters from this notebook (instead of the Excel template)
- Fill in the missing fields in the `key_outputs`, `decision_maker_options` and the `theme_weights`.
- All the input parameters are stored in the `dict` object `case.input_dict` created when the case was built.
- Hint: You can access the `key_outputs` through `case.input_dict['key_outputs']`

In [ ]:
import numpy as np
def setParameterValue(sheet_name, first_param, second_param, value):
    if sheet_name == 'key_outputs':
        index = np.where(case.input_dict[sheet_name] == first_param)
        case.input_dict["_".join([sheet_name[:-1], second_param])][index] = value
        
    if sheet_name == 'decision_makers_options':
        index1 = np.where(case.input_dict[sheet_name] == first_param)
        index2 = np.where(case.input_dict['internal_variable_inputs'] == second_param)
        case.input_dict['decision_makers_option_value'][index1, index2] = value

In [ ]:
setParameterValue('key_outputs', 'Income growth for low-income families', 'theme', 'Reduced Inequality')
setParameterValue('decision_makers_options', 'Focus on cryptocurrency', 'Investment in cryptocurrency', 550000)

### Calculate key output values

Now that we have a decent set-up of our case, we can `evaluate` it. Cases are evaluated using the following iterative four-step process.

1. `evaluate_single_dependency` is the smallest calculation step. It takes both arguments from a **single dependency row** and applies the provided operator. This is calculated for a fixed decision makers option (dmo) and fixed scenario. <br>
_e.g. this function will calculate 'Effect of training' as 'Investment' / 'Effectiveness'. The fixed decision makers option and fixed scenario is used (when needed) to find the correct values for the internal variable ('Investment') and external variable ('Effectivess')._
<br>

2. `evaluate_all_dependencies` calculates all singular calculated dependencies to evaluate the full outcome for a case for a fixed decision makers option and fixed scenario. <br>
_e.g. this function will calculate the values of the key outputs for the 'Base case' with a 'Focus on clean drinking water'._
<br>

3. `evaluate_selected_scenario` calculates all full outcomes for a case for **all decision makers options** and single given scenario. <br>
_e.g. this function will calculate a set of values of the key outputs for the 'Base case' for each option that is available to the decision makers, that is, one set for 'Focus on water recycling', one set for 'Focus on training' and one set for 'Equal spread'._
<br>

4. `evaluate_all_scenarios` calculates all sets of key output values for **all decision makers options** and **all scenarios**. 

The process is iterative. So, when calling `evaluate` step 4 is called that will call the other functions. This process is visualised below:

![Process Visual](images/process_visual.png)

In [ ]:
case.evaluate()

The `evaluate` function creates an output dictionary (`output_dict`) for our `case`. This dictionary contains all the results of our case and is structured as a nested dictionary with the following structure: 

{'Scenario A': <br>
   { <br>
    'Decision Makers Option I': <br>
        {<br>
        'key_outputs': {'Key Output X': value, ..}<br>
        }<br>
    'Decision Makers Option II': {..}<br>
    }<br>
'Scenario B': {..}<br>
}

In [ ]:
case.output_dict['Base case']

### Process key output values
Now that we have evaluated the dependencies, we can start to compare the key outputs. In order to do this we `appreciate` the key output values in our case. A five-step approach is introduced when appreciating the outputs:
1. `appreciate_single_key_output` transforms a key output value into an **appreciated value**. This is done by a (non)-linear appreciation based on the minimum and maximum values of that specific key output throughout the case (so based on values of key outputs in all scenarios and decision makers options). For each (scenario, decision makers option) combination
<br>

2. `apply_weights` applies theme and key output weights to the calculated appreciations under Step 1. These weights are model inputs determined based on strategic priorities. This function gives us a set of **weighted appreciations** of the key outputs for each (scenario, decision makers option) combination. 
<br>

3. (**NOT YET IMPLEMENTED**) `decision_makers_option_appreciations` aggregates the weighted appreciations of key outputs into a single **aggregated appreciation** per decision makers option and scenario.
<br>

4. (**NOT YET IMPLEMENTED**) `apply_scenario_weights` applies scenario weights to the calculated appreciation under Step 3. This function gives us a value of **scenario weighted appreciations** for each decision makers option and scenario.
<br>

5. (**NOT YET IMPLEMENTED**) `scenario_appreciations` aggregates the scenario weighted appreciations into a single value for each decision makers option.

In [ ]:
case.appreciate()

The `appreciate` function extends our existing output dictionary with information about the appreciation & weighted appreciation. The extended structure looks as follows:

{'Scenario A': <br>
   { <br>
    'Decision Makers Option I': <br>
        {<br>
        'key_outputs': {'Key Output X': value, ..}<br>
        **'appreciations': {'Key Output X': value, ..}<br>
        'weighted_appreciations': {'Key Output X': value, ..}**<br>
        }<br>
    'Decision Makers Option II': {..}<br>
    }<br>
'Scenario B': {..}<br>
}

In [ ]:
case.output_dict['Pessimistic']

### Visualization of output
Having evaluated and appreciated the full case, it is time to start creating some visuals. We will do this using the generated `output_dict` and the `visualize` function. This function uses a display format (currently 'table' or 'barchart') and the key of the result as stored in `output_dict` (e.g. `key_outputs` or `weighted_appreciations`).

Visualizations are generated using the following rules:

- `table`: 2- or 3-dimensional tables. Rows contain scenarios and/or key outputs (both if 3-dimensional, choose one if 2-dimensional). Columns are decision makers options. 
- `barchart`:
    - 3-dimensional is not possible. Specify a scenario if the `key` in the `output_dict` is associated with three dimensions
    - 1 or 2-dimensional: decision makers options are always on the horizontal axis. For 2-dimensional barchart it can be either `stacked` or `dodged`. 

In [ ]:
case.visualize('table', 'key_outputs', scenario='Base case')

Comparing key outputs for each decision makers option can become difficult: the ranges of the three key outputs vary wildly. For better comparisons, we can therefore look at the weighted appreciations:

In [ ]:
case.visualize('table', 'weighted_appreciations', scenario="Base case")

In [ ]:
case.visualize(visual_request='barchart', key='weighted_appreciations', scenario="Base case")

### (Ideas for extra data science)
- Change to Nyenrode colors
- Add labels to the grpahs. Add the number/total appreciation value
- Use an alternative way of visualization
- Subplots with all the cases next to each other
- 

### Transforming the case into a different format (optional)
We can transform a case into a different format. The `transform` function creates a folder named as the file extension and stores this at the provided location (here: in the folder where the repo is stored)

In [52]:
output_path = Path.cwd().parent.parent
case.transform(output_path, "json")
# case.transform(output_path, "xlsx")
case.transform(output_path, "csv")

Folder 'json' already exists. 
Folder 'csv' already exists. 
